In [ ]:
from hyrax import Hyrax

h = Hyrax(config_file='./astrominn_testing_runtime_config.toml')
h.config['train']['epochs'] = 1

In [ ]:
ds = h.prepare()
ds[0]

In [ ]:
m = h.train()

## Below is investigation into ONNX-ification

In [ ]:
from hyrax.model_exporters import export_to_onnx
from hyrax.pytorch_ignite import dist_data_loader
from pathlib import Path
h.set_config('data_loader.batch_size', 1)
context = {
    "ml_framework": "pytorch",
    "results_dir": Path('./'),
}

data_loaders = dist_data_loader(ds, h.config, ["train", "validate"])
train_data_loader, _ = data_loaders['train']

sample = m.to_tensor(next(iter(train_data_loader)))
m.train(False)
m.to("cpu")
m(sample)

export_to_onnx(m, sample, h.config, context)


In [ ]:
sample = m.to_tensor(next(iter(train_data_loader)))

m(sample)

In [ ]:
import onnxruntime
ort_session = onnxruntime.InferenceSession('/Users/drew/code/applecider/notebooks/testing/example_model_opset_20.onnx')
samps = [s.detach().cpu().numpy() for s in sample]
ort_inputs = {ort_session.get_inputs()[0].name: samps[0],
                ort_session.get_inputs()[1].name: samps[1]}
ort_session.run(None, ort_inputs)